# Metatranscriptome rRNA depletion and Library Prep Test
### Yike Shen, Tessa R. Bloomquist, Abigail Rauso, Peter Meyn, Andrea A. Baccarelli
### 12/06/2021

YS, TRB, AR, AAB - Department of Environmental Health Sciences, Columbia University\
PM - Genome Technology Center, New York University\
Corresponding to: Yike Shen, yike.shen@columbia.edu

Due to ongoing challenging in rRNA depletion kits in the market in human stool shotgun metatranscriptome sequencing. We tested the following combinations of rRNA depletion and library prepration:
- Illumina rRNA Depletion + Illumina Library Prepration
- Illumina rRNA Depletion + NEB Library Prepration
- NEB rRNA Depletion + Illumina Library Prepration
- NEB rRNA Depletion + NEB Library Prepration

Kits link:
- Illumina Ribo-Zero Plus rRNA Depletion Kit: https://www.illumina.com/products/by-type/accessory-products/ribo-zero-plus-rrna-depletion.html
- NEBNext® rRNA Depletion Kit (Bacteria): https://www.neb.com/products/e7850-nebnext-rrna-depletion-kit-bacteria#Product%20Information
- Illumina Library Prepration (TruSeq Stranded mRNA): https://www.illumina.com/products/by-type/sequencing-kits/library-prep-kits/truseq-stranded-mrna.html
- NEBNext® Ultra™ II Directional RNA Library Prep Kit for Illumina®: https://www.neb.com/products/e7760-nebnext-ultra-ii-directional-rna-library-prep-kit-for-illumina#Product%20Information

### 1.0 Sequencing depth

896M	IllDep-IllPrep-A_R1.fastq.gz\
943M	IllDep-IllPrep-A_R2.fastq.gz\
957M	IllDep-IllPrep-B_R1.fastq.gz\
987M	IllDep-IllPrep-B_R2.fastq.gz\
663M	IllDep-IllPrep-C_R1.fastq.gz\
715M	IllDep-IllPrep-C_R2.fastq.gz\
1.9G	IllDep-NEBPrep-A_R1.fastq.gz\
2.0G	IllDep-NEBPrep-A_R2.fastq.gz\
1.9G	IllDep-NEBPrep-B_R1.fastq.gz\
2.0G	IllDep-NEBPrep-B_R2.fastq.gz\
1.2G	IllDep-NEBPrep-C_R1.fastq.gz\
1.3G	IllDep-NEBPrep-C_R2.fastq.gz\
680M	NEBDep-IllPrep-A_R1.fastq.gz\
705M	NEBDep-IllPrep-A_R2.fastq.gz\
712M	NEBDep-IllPrep-B_R1.fastq.gz\
739M	NEBDep-IllPrep-B_R2.fastq.gz\
876M	NEBDep-IllPrep-C_R1.fastq.gz\
911M	NEBDep-IllPrep-C_R2.fastq.gz\
1.9G	NEBDep-NEBPrep-A_R1.fastq.gz\
2.0G	NEBDep-NEBPrep-A_R2.fastq.gz\
1.7G	NEBDep-NEBPrep-B_R1.fastq.gz\
1.8G	NEBDep-NEBPrep-B_R2.fastq.gz\
1.8G	NEBDep-NEBPrep-C_R1.fastq.gz\
1.9G	NEBDep-NEBPrep-C_R2.fastq.gz\

Note: We allocated small reads to the test, the reads will be much deeper in a larger flowcell

### 2.0 Kneaddata to do metatranscriptome QC
pseudocode example below, follow bioBakery Kneaddata tutorial to install dependencies

In [ ]:
#!/bin/bash
#SBATCH --time=30:00:00
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=100G
#SBATCH --job-name=$jobname

export OMP_NUM_THREADS=4
conda init bash
export CONDA3PATH=/$path/anaconda3
module load Conda/3
conda activate biobakery3

kneaddata --input /$path/$Sample_R1.fastq.gz \
--input /$path/$Sample_R2.fastq.gz \
--reference-db /$path/bowtie2index \
--reference-db /$path/bowtie2transcriptome \
--reference-db /$path/silvadb \
--output /$path/kneaddataOutputPairedEnd_$sample \
--trimmomatic /opt/software/Trimmomatic/0.39-Java-1.8/ \
--trimmomatic-options="ILLUMINACLIP:/opt/software/Trimmomatic/0.39-Java-1.8/adapters/NexteraPE-PE.fa:2:30:10"

conda deactivate

scontrol show job $SLURM_JOB_ID

#Note, one kneaddata step is very long, I would recommend request at least 30 hours job run time
#Note, preferable to run in scartch space, the mid files are very large, at least 20 times than the initial zipped size

### 3.0 Run fastqc & multiqc 

In [ ]:
mkdir $directory
export PATH=$PATH:/$path/FastQC/
ls *fastq|while read line; do fastqc -o $directory/ -t 28 "$line";done

multiqc .

### 4.0 Results

In [1]:
import pandas as pd
import numpy as np
pd.read_excel('metatranscriptome_test_results_Shen.xlsx', sheet_name='2021',header=1)

,Sample,Raw reads (million),minus adapter (million),rRNA contamination (million),Final reads (million),Effective reads w duplication (%),rRNA (%) to total,rRNA (%) to adapter removed,duplication rate (%),Final non duplicated reads(million),Final non duplicated %
0,IllDepIllPrep_A,18.7,18.0,13.7,3.5,19.444444,73.262032,76.111111,67.4,1.1410,6.338889
1,IllDepNEBPrep_A,40.4,38.6,28.0,9.4,24.352332,69.306931,72.538860,67.4,3.0644,7.938860
2,NEBDepIllPrep_A,14.3,13.8,11.0,2.4,17.391304,76.923077,79.710145,76.0,0.5760,4.173913
3,NEBDepNEBPrep_A,40.9,39.5,33.4,5.4,13.670886,81.662592,84.556962,60.0,2.1600,5.468354
4,IllDepIllPrep_B,17.4,16.8,7.7,8.6,51.190476,44.252874,45.833333,43.0,4.9020,29.178571
5,IllDepNEBPrep_B,34.2,33.1,12.8,19.6,59.214502,37.426901,38.670695,47.0,10.3880,31.383686
6,NEBDepIllPrep_B,13.4,13.0,7.9,4.8,36.923077,58.955224,60.769231,46.0,2.5920,19.938462
7,NEBDepNEBPrep_B,31.8,30.8,17.0,13.3,43.181818,53.459119,55.194805,40.0,7.9800,25.909091
8,IllDepIllPrep_C,16.0,15.5,12.8,2.0,12.903226,80.000000,82.580645,66.0,0.6800,4.387097
9,IllDepNEBPrep_C,32.2,31.2,25.8,4.5,14.423077,80.124224,82.692308,67.0,1.4850,4.759615


### 5.0 Conclusions
- NEB kit and illumina kit performs similar in rRNA depletion. Both not depleting well. 
- NEB library prep sequences much deeper than illumina library prep. If the allocated sequence deeper, illumina library prep was also able to sequence deep. 
- % of effective reads is sample dependent (B performs best)
- rRNA depletion is still challenging, one option we decided is to do very deep sequencing and bioinformatically remove rRNA and other contamination (host and adapters).